In [9]:
import json

# read the dataset
# please enter the path of your data
split = 'all'
data_path = './data/Product/all/' + split + '.jsonl'
docs = []
with open(data_path) as f:
    for line in f:
        docs.append(json.loads(line))
n_meetings = len(docs)
print('Total {} meetings in the {} set.'.format(n_meetings, split))

Total 20 meetings in the test set.


In [10]:
import re
# filter some noises caused by speech recognition
def clean_data(text):
    text = text.replace('{vocalsound} ', '')
    text = text.replace('{disfmarker} ', '')
    text = text.replace('a_m_i_', 'ami')
    text = text.replace('l_c_d_', 'lcd')
    text = text.replace('p_m_s', 'pms')
    text = text.replace('t_v_', 'tv')
    text = text.replace('{pause} ', '')
    text = text.replace('{nonvocalsound} ', '')
    text = text.replace('{gap} ', '')
    return text

In [11]:
import pandas as pd
import nltk

# Create a dictionary to store the data for train dataframe
train_data = {'meeting_id': [], 'caption': [], 'speaker': [], 'st': [], 'en': []}

# Create a dictionary to store the data for test dataframe
test_data = {'meeting_id': [], 'st': [], 'en': [], 'topic': []}
meeting_id = 0

for data in docs:
    # Loop through each meeting transcript
    # we can assume that approximately 1,000 characters are spoken in one minute of an audio file.
    # so we get 60 millisecond for 1 character
    millis_per_character = 60
    millis_per_switch = 1000
    fake_times = {}
    time_millis = 0
    for i, transcript in enumerate(data['meeting_transcripts']):
        speaker = transcript['speaker']
        caption = transcript['content']
        fake_times[i] = time_millis
        # for s in sentences:
        train_data['meeting_id'].append(meeting_id)
        train_data['caption'].append(caption)
        train_data['speaker'].append(speaker)

        train_data['st'].append(time_millis)
        time_millis += len(caption) * millis_per_character
        train_data['en'].append(time_millis)


# Loop through each topic and its relevant text spans
    for j, topic in enumerate(data['topic_list']):
        for span in topic['relevant_text_span']:
            st, en = map(int, span)
            test_data['meeting_id'].append(meeting_id)
            test_data['topic'].append(topic['topic'])
            test_data['st'].append(fake_times[st])
            test_data['en'].append(fake_times[en])
    meeting_id = meeting_id + 1  # Meeting id starts from 1



# Create train and test dataframes
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data).sort_values(by=['meeting_id', 'st'])

In [12]:
train_df.head()

,meeting_id,caption,speaker,st,en
0,0,"Here we go . Welcome everybody . Um , I'm Abig...",Project Manager,0,17700
1,0,"Hi I'm Chiara , I'm the um Marketing Expert . ...",Marketing,17700,29160
2,0,"I think we'll get around to that , yeah . So t...",Project Manager,29160,33840
3,0,We'll get round to that later . My name is Chi...,Marketing,33840,38640
4,0,Okay . I forgot to s say I'm the Project Manag...,Project Manager,38640,44040


In [13]:
train_df.value_counts(['speaker'])

speaker            
Project Manager        3345
Industrial Designer    2900
User Interface         2769
Marketing              2372
dtype: int64

In [14]:
test_df.head()

,meeting_id,st,en,topic
0,0,53220,373560,Tool training on the conceptual design of remo...
1,0,322140,780000,Project plan and target revenue
2,0,373560,823080,Brainstorming functional design on technical f...
3,1,0,152700,Introduction of a prototype
4,1,153420,212100,Evaluation criteria of remote control


In [15]:
train_df.to_csv('train_ami.csv', index=False)
test_df.to_csv('test_ami.csv', index=False)